# Dictionary Learning

Solve orthogonal dictionary learning problem taken from: Yu Bai, Qijia Jiang, and Ju Sun. 
["Subgradient descent learns orthogonal dictionaries."](https://arxiv.org/abs/1810.10702) arXiv preprint arXiv:1810.10702 (2018).

## Problem Description

Given data $\{y_i \}_{i \in[m]}$ generated as $y_i = A x_i$, where $A \in R^{n \times n}$ is a fixed unknown orthogonal matrix and each $x_i \in R^n$ is an iid Bernoulli-Gaussian random vector with parameter $\theta \in (0,1)$, recover $A$. 

Write $Y \doteq [y_1,...,y_m]$ and $X \doteq [x_1,...,x_m]$. To find the column of $A$, one can perform the following optimization:

$$\min_{q \in R^n} f(q) \doteq \frac{1}{m} ||q^T Y||_{1} = \frac{1}{m} \sum_{i=1}^m |q^T y_i|,$$
$$\text{s.t.} ||q||_2 = 1$$

This problem is nonconvex due to the constraint and nonsmooth due to the objective.

Based on the above statistical model, $q^T Y = q^T A X$ has the highest sparsity when $q$ is a column of $A$ (up to sign) so that $q^T A$ is 1-sparse. 

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import numpy as np
import torch
import numpy.linalg as la
from scipy.stats import norm
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

from pygranso.private.getNvar import getNvarTorch
import torch.nn as nn

## Initialization 
Specify torch device, create torch model and generate data

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device('cuda')

class Dict_Learning(nn.Module):
    
    def __init__(self,n):
        super().__init__()
        np.random.seed(1)
        q0 = norm.ppf(np.random.rand(n,1))
        q0 /= la.norm(q0,2)
        self.q = nn.Parameter( torch.from_numpy(q0) )
    
    def forward(self, Y,m):
        qtY = self.q.T @ Y
        f = 1/m * torch.norm(qtY, p = 1)
        return f

## Data initialization
n = 30
np.random.seed(1)
m = 10*n**2   # sample complexity
theta = 0.3   # sparsity level
Y = norm.ppf(np.random.rand(n,m)) * (norm.ppf(np.random.rand(n,m)) <= theta)  # Bernoulli-Gaussian model
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
Y = torch.from_numpy(Y).to(device=device, dtype=torch.double)

torch.manual_seed(0)

model = Dict_Learning(n).to(device=device, dtype=torch.double)

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
def user_fn(model,Y,m):
    # objective function    
    f = model(Y,m)

    q = list(model.parameters())[0]

    # inequality constraint
    ci = None

    # equality constraint 
    ce = pygransoStruct()
    ce.c1 = q.T @ q - 1

    return [f,ci,ce]

comb_fn = lambda model : user_fn(model,Y,m)

## User Options
Specify user-defined options for PyGRANSO

In [4]:
opts = pygransoStruct()
opts.torch_device = device
opts.maxit = 500
np.random.seed(1)
nvar = getNvarTorch(model.parameters())
opts.x0 = torch.nn.utils.parameters_to_vector(model.parameters()).detach().reshape(nvar,1)

opts.print_frequency = 10

## Main Algorithm

In [5]:
start = time.time()
soln = pygranso(var_spec= model, combined_fn = comb_fn, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(max(abs(soln.final.x))) # should be close to 1



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.0.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021 Tim Mitchell and Buyun Liang                                       ║ 
═════════════════════════════════════════════